In [2]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import re

def load_pfm(file):
    with open(file, 'rb') as f:
        # Read the header
        header = f.readline().rstrip()
        color = False
        if header == b'PF':
            color = True
        elif header != b'Pf':
            raise Exception('Not a PFM file.')

        # Read the dimensions
        dim_match = re.match(rb'^(\d+)\s(\d+)\s$', f.readline())
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception('Malformed PFM header.')

        # Read the scale
        scale = float(f.readline().rstrip())
        endian = '<' if scale < 0 else '>'  # little or big endian
        scale = abs(scale)

        # Read the data
        data = np.fromfile(f, endian + 'f')
        shape = (height, width, 3) if color else (height, width)
        data = np.reshape(data, shape)
        data = np.flipud(data)  # PFM files are stored in top-to-bottom order
        return data, scale

def convert_pfm_to_png(pfm_folder, png_folder):
    if not os.path.exists(png_folder):
        os.makedirs(png_folder)

    for filename in os.listdir(pfm_folder):
        if filename.endswith('.pfm'):
            pfm_path = os.path.join(pfm_folder, filename)
            disparity_data, _ = load_pfm(pfm_path)

            # Normalize the disparity for visualization
            vmin = np.percentile(disparity_data, 5)
            vmax = np.percentile(disparity_data, 95)

            # Create the color map
            plt.imshow(disparity_data, cmap='viridis', vmin=vmin, vmax=vmax)

            # Remove the axis
            plt.axis('off')

            # Save the image
            png_path = os.path.join(png_folder, os.path.splitext(filename)[0] + '.png')
            plt.savefig(png_path, bbox_inches='tight', pad_inches=0)
            plt.close()  # Close the figure to free memory

# Usage
pfm_folder = '/content/drive/MyDrive/UT-MedRobo/Codebases/STTR/result_disp'
png_folder = '/content/drive/MyDrive/UT-MedRobo/Codebases/STTR/result_disp_vis'
convert_pfm_to_png(pfm_folder, png_folder)